In [1]:
import json
import warnings
import pandas as pd
pd.options.mode.chained_assignment = None
from sec_api import QueryApi, XbrlApi

In [80]:
api_key = "e3d46df2271d81a4b7a71153aef74fa5e7dae956314122b155767267425b36a6"

# Query 20 most recent 10Q Info

In [ ]:
# queryAPI = QueryApi(api_key=api_key)

In [ ]:
# query = {"query": {
#             "query_string": {
#                 "query": "ticker:TSLA AND formType:\"10-Q\"",
#                     }
#                 },
#          "from": "0",
#          "size": 4*5,
#          "sort": [{"filedAt": {"order": "desc"}}]
#             }

In [ ]:
# response = queryAPI.get_filings(query)
# response_parsed = pd.DataFrame.from_records(response["filings"])

In [74]:
# response_parsed.to_csv("TSLA_10Q_INFO_2017Q1_2023Q2.csv", index=False)
response_parsed = pd.read_csv("TSLA_10Q_INFO_2017Q1_2023Q2.csv")

# Parse 10Q

In [81]:
# xbrlAPI = XbrlApi(api_key)

In [94]:
# %%time
# for date, url in response_parsed[["periodOfReport", "linkToFilingDetails"]].values:
#     xbrl_json = xbrlAPI.xbrl_to_json(url)
    
#     with open(f"tesla_10Q_{date}.json", "w") as f:
#         json.dump(xbrl_json, f)

CPU times: total: 4.41 s
Wall time: 10.9 s


## Extract Data

In [2]:
with open("tesla_10Q_2023-06-30.json", "r") as f:
    xbrl_json = json.load(f)

In [62]:
current_date = xbrl_json["CoverPage"]["DocumentPeriodEndDate"]
current_date

'2023-06-30'

In [4]:
BalanceSheets = xbrl_json["BalanceSheets"]
StatementsOfIncome = xbrl_json["StatementsOfIncome"]
StatementsOfCashFlows = xbrl_json["StatementsOfCashFlows"]
StatementsOfShareholdersEquity = xbrl_json["StatementsOfShareholdersEquity"]

## Balance Sheets

In [20]:
def balancesheets_to_df(BalanceSheets):
    """
    """
    df_list = []

    for name in BalanceSheets.keys():
        try:
            df = pd.DataFrame.from_records(BalanceSheets[name])
            df["period"] = df["period"].apply(lambda x: x["instant"])
            df["name"] = name
            df_list.append(df)
        except Exception as ex:
            print(name, "---", ex)
    
    return pd.concat(df_list, axis=0)

In [21]:
balancesheet_df = balancesheets_to_df(BalanceSheets)
set(BalanceSheets.keys()).difference(set(balancesheet_df.name))

set()

In [47]:
(balancesheet_df.query("segment.isna()").value_counts(["name", "period"]) == 1).mean()

1.0

In [64]:
# balancesheet_df.query("segment.isna() and period == @current_date")

## Statements of Income

In [55]:
def incomestatement_to_df(StatementsOfIncome):
    """
    """
    df_list = []

    for name in StatementsOfIncome.keys():
        try:
            df = pd.DataFrame.from_records(StatementsOfIncome[name])
            df["startDate"] = df["period"].apply(lambda x: x["startDate"])
            df["endDate"] = df["period"].apply(lambda x: x["endDate"])
            df["name"] = name
            df_list.append(df)
        except Exception as ex:
            print(name, "---", ex)
    
    return pd.concat(df_list, axis=0)

In [56]:
income_df = incomestatement_to_df(StatementsOfIncome)
set(StatementsOfIncome.keys()).difference(set(income_df.name))

set()

In [60]:
(income_df.query("segment.isna()").value_counts(["name", "startDate", "endDate"]) == 1).mean()

1.0

In [69]:
(income_df.query("segment.isna()").value_counts(["endDate", "name"]) == 2).mean()

1.0

In [72]:
# (income_df.query("segment.isna() and endDate == @current_date")
#  .sort_values(["name", "startDate"]).groupby(["name"]).tail(1))